Preprocessing of the data

In [ ]:
# 1. Prepare bed files with coordinates

def prepare_bed_files(de_file, FDR, region, dist_from_TSS):
    
    m = open('DeSeq2_'+de_file+'.csv', 'r').readlines()
    
    de_up = [l.strip().split('\t')[0][0:15] for l in m[1:] if l.strip().split('\t')[6]!='NA' 
          and float(l.strip().split('\t')[6])<FDR 
         and float(l.strip().split('\t')[2])>0] # positive log2FC (=higher expression) in the first grade of the two compared
    de_down = [l.strip().split('\t')[0][0:15] for l in m[1:] if l.strip().split('\t')[6]!='NA' 
          and float(l.strip().split('\t')[6])<FDR 
         and float(l.strip().split('\t')[2])<0] # negative log2FC (=lower expression) in the first grade of the two compared
    g = open('hg38_'+region+'_'+de_file+'_up.bed', 'w')
    h = open('hg38_'+region+'_'+de_file+'_down.bed', 'w')
    if region == 'promoters':
        f = open('hg38_genes.bed', 'r').readlines()
        for line in f:
            line = line.strip().split('\t')
            if line[3] in de_up: # check whether the gene is Differentially Expressed
                if line[5]=='+':
                    g.write(line[0]+'\t'+str(int(line[1])-dist_from_TSS)+'\t'+str(int(line[1])+dist_from_TSS)+'\t'+line[3]+'\t'+line[4]+'\t'+line[5]+'\n')
                if line[5]=='-':
                    g.write(line[0]+'\t'+str(int(line[2])-dist_from_TSS)+'\t'+str(int(line[2])+dist_from_TSS)+'\t'+line[3]+'\t'+line[4]+'\t'+line[5]+'\n')
            if line[3] in de_down:
                if line[5]=='+':
                    h.write(line[0]+'\t'+str(int(line[1])-dist_from_TSS)+'\t'+str(int(line[1])+dist_from_TSS)+'\t'+line[3]+'\t'+line[4]+'\t'+line[5]+'\n')
                if line[5]=='-':
                    h.write(line[0]+'\t'+str(int(line[2])-dist_from_TSS)+'\t'+str(int(line[2])+dist_from_TSS)+'\t'+line[3]+'\t'+line[4]+'\t'+line[5]+'\n')
    if region == 'enhancers':
        f = open('hg38_enhancers.bed', 'r').readlines()
        for line in f:
            line = line.strip().split('\t')
            if line[3] in de_up: 
                g.write('\t'.join(line)+'\n')
            if line[3] in de_down:
                h.write('\t'.join(line)+'\n')
prepare_bed_files('grade1_grade23', 0.01, 'enhancers', 1000)


In [ ]:
# Run bedtools to generate fasta files with the desired sequences
import os

def get_fasta_files(de_file, region, up_or_down):
    comm = os.popen('bedtools getfasta -fi hg38.fa -bed hg38_'+region+'_'+de_file+'_'+up_or_down+'.bed -fo '+de_file+'_'+region+'_seq_'+up_or_down+'.fa')
    comm.read()
    comm.close()
#get_fasta_files('grade1_grade23', 'promoter', 'up')
get_fasta_files('grade1_grade23', 'enhancers', 'down')

In [ ]:
import numpy as np
import h5py
############################################
### Function to load data in h5py format ###
############################################

'''def load_data(path_to_data):
    data = h5py.File(path_to_data,'r')
    X_test_seq = np.transpose(np.array(data['test_in_seq']),axes=(0,2,1))
    X_test_region = np.transpose(np.array(data['test_in_region']),axes=(0,2,1))
    y_test_RBP = np.array(data['test_out'])
    y_test_name= np.array(data['test_name'])
    y_train= np.array(data['train_out'])
    data.close()

    return X_test_seq, X_test_region, y_test_RBP, y_test_name, y_train'''

In [164]:
# Split the promoter sequences into shorter sequences, which will be the input for the model.
def split_into_short_sequences(de_file, region, up_or_down, short_seq_length, step):
    long_sequence_file = open(de_file+'_'+region+'_seq_'+up_or_down+'.fa', 'r').readlines()
    short_seq_list = []
    for long_seq in long_sequence_file:
        if '>' not in long_seq:
            long_seq = long_seq.strip()
            for x in range(0, len(long_seq), step): #97?
                short_seq = long_seq[x:x+short_seq_length]
                #print(short_seq)
                short_seq_list.append(short_seq)
    return short_seq_list
grade1_23_promoter_up = split_into_short_sequences('grade1_grade23', 'promoter', 'up', 150, 20)
grade1_23_promoter_down = split_into_short_sequences('grade1_grade23', 'promoter', 'down', 150, 20)

In [165]:
len(grade1_23_promoter_up)
grade1_23_promoter_up = grade1_23_promoter_up[0:50] #I'm using less sequences just for the sake of speed of testing
len(grade1_23_promoter_up)
grade1_23_promoter_down = grade1_23_promoter_down[0:50]

In [166]:
##########################################################
### Function to generate random 'region' sequences     ###
##########################################################

def generate_random_regions(seq_len, nr_of_seqs): # Generate random regions, just for testing
    test_in_region = []
    for nr in range(nr_of_seqs):
        single_reg = ''.join([random.choice(['i', '3', '5']) for i in range(seq_len)])
        test_in_region.append(single_reg)
    return test_in_region

grade1_23_promoter_up_region = generate_random_regions(250, 50)
grade1_23_promoter_down_region = generate_random_regions(250, 50)
#print(grade1_23_promoter_up_region)


In [ ]:

##########################################################
### Function to generate random nucleotide sequences   ###
##########################################################

'''def generate_random_sequence(seq_len, nr_of_seqs): # Generate random sequence, just for testing
    test_in_seq = []
    for nr in range(nr_of_seqs):
        single_seq = ''.join([random.choice(['A', 'T', 'G', 'C', 'a', 't', 'g', 'c']) for i in range(seq_len)])
        test_in_seq.append(single_seq)
    return test_in_seq

test_in_seq_rand = generate_random_sequence(150, 15) # Te liczby tzn 150 i 250 dla regions musza takie byc inaczej models.py nie hula.

train_in_seq_rand = generate_random_sequence(150, 15)
print(train_in_seq_rand)
valid_in_seq_rand = generate_random_sequence(150, 15)
#train_in_region = generate_random_regions(250, 15)
#valid_in_region = generate_random_regions(250, 15)'''

In [167]:
import numpy as np
import h5py


##########################################################
### Function to convert the sequence to one-hot encode ### # taken from Deep-Ripe scripts
##########################################################

def seq_to_mat(seq):
    seq_len = len(seq)
    seq = seq.replace('A','0')
    seq = seq.replace('a','0')
    seq = seq.replace('C','1')
    seq = seq.replace('c','1')
    seq = seq.replace('G','2')
    seq = seq.replace('g','2')
    seq = seq.replace('T','3')
    seq = seq.replace('t','3')
    seq = seq.replace('U','3')
    seq = seq.replace('u','3')
    seq = seq.replace('N','4') #some cases have N in sequence
    seq = seq.replace('n','4')
    seq_code = np.zeros((4,seq_len), dtype='float16')
    for i in range(seq_len):
        if int(seq[i]) != 4:
            seq_code[int(seq[i]),i] = 1
        else:
            seq_code[0:4,i] = np.tile(0.25,4)
    return np.transpose(seq_code)

##########################################################
### Function to convert the region to one-hot encode ###
##########################################################

def region_to_mat(region):
    region_len = len(region)
    region= region.replace('i','0')
    region= region.replace('c','1')
    region= region.replace('3','2')
    region= region.replace('5','3')
    region= region.replace('N','4')
    region_code = np.zeros((4,region_len), dtype='float16')
    for i in range(region_len):
        if int(region[i]) != 4:
            region_code[int(region[i]),i] = 1
        else:
            region_code[0:4,i] = np.tile(0.25,4)
    return np.transpose(region_code)
import random
import h5py
import string

In [168]:
##############################################################
### Function to split the sequences into 4 separate arrays ###
##############################################################
#print(test_in_seq)
#print(np.array(test_in_seq_rand))
def split_into_4_arrays(input_list):
    for j in range(len(input_list)):
        sekw = input_list[j].strip()
        if sekw[0] in ['A', 'C', 'G', 'T', 'U', 'a', 'c', 'g', 't', 'u']:
            sekw = seq_to_mat(sekw)
        elif sekw[0] in ['i', '3', '5']:
            sekw = region_to_mat(sekw)
        sekw = np.array([sekw[:,0], sekw[:,1], sekw[:,2], sekw[:,3]])
        input_list[j] = sekw
    result = np.array(input_list)
    return result

#test_in_seq1 = split_into_4_arrays(['AAAGGGCCCTTTTT', 'AAAGGGCCCGGGGG'])
grade1_23_promoter_up_one_hot = split_into_4_arrays(grade1_23_promoter_up)
grade1_23_promoter_down_one_hot =  split_into_4_arrays(grade1_23_promoter_down)
grade1_23_promoter_up_one_hot_region = split_into_4_arrays(grade1_23_promoter_up_region)
grade1_23_promoter_down_one_hot_region =  split_into_4_arrays(grade1_23_promoter_down_region)
#test_name = np.array([random.choice(string.ascii_letters) for i in range(len(test_in_seq))], dtype='S')
#print(type(test_in_seq1))
#print(type(np.array(test_in_seq_rand)))

In [169]:
print(grade1_23_promoter_up_one_hot.shape)
print(len(grade1_23_promoter_up_one_hot))
print(grade1_23_promoter_up_one_hot_region.shape)
print(len(grade1_23_promoter_up_one_hot_region))
print(grade1_23_promoter_up_one_hot_region)

(50, 4, 150)
50
(50, 4, 250)
50
[[[0. 1. 1. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 1.]]

 [[1. 1. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 1. 1. 1.]]

 ...

 [[0. 1. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [1. 0. 0. ... 1. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 1. 1. 0.]
  [1. 1. 1. ... 0. 0. 1.]]

 [[1. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 1. 1.]]]


In [170]:
#### Create labels from the data

from keras.utils import to_categorical
import numpy as np

labels = ['0' for lab in range(len(grade1_23_promoter_up_one_hot))] +\
['1' for lab in range(len(grade1_23_promoter_down_one_hot))]
labels = np.array(labels)
print(labels)
# one hot encoded labels
labels = to_categorical(labels)
print(labels)


['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1']
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]

In [171]:
### Splitting the data into training 70%, validation 20% and testing 10%

import numpy
numpy.random.seed(0)
liczba_do_losowania = len(grade1_23_promoter_up_one_hot) + len(grade1_23_promoter_down_one_hot)
indices = np.random.permutation(liczba_do_losowania)

indices_train = indices[0:int(liczba_do_losowania*0.7)]
indices_valid = indices[int(liczba_do_losowania*0.7) : int(liczba_do_losowania*0.9)]
indices_test = indices[int(liczba_do_losowania*0.9):]

data = np.concatenate((grade1_23_promoter_up_one_hot, grade1_23_promoter_down_one_hot), axis=0)
#print(grade1_23_promoter_up_one_hot)
#print(indices_train)
#print(data[3,])
train_in_seq = data[indices_train]
valid_in_seq = data[indices_valid]
test_in_seq = data[indices_test]

train_out = labels[indices_train]
valid_out = labels[indices_valid]
test_out = labels[indices_test]
print(test_in_seq.shape)
print(valid_in_seq.shape)
print(train_in_seq.shape)
print(train_out.shape)

(10, 4, 150)
(20, 4, 150)
(70, 4, 150)
(70, 2)


In [173]:
region = np.concatenate((grade1_23_promoter_up_one_hot_region, grade1_23_promoter_down_one_hot_region), axis=0)
train_in_region = region[indices_train]
valid_in_region = region[indices_valid]
test_in_region = region[indices_test]

In [174]:
test_name = np.array([random.choice(string.ascii_letters) for i in range(len(test_in_seq))], dtype='S')

In [175]:
#######################################################
### Function to generate random labels dataset      ###
#######################################################

'''def generate_out(nr_of_seqs):
    result = []
    for k in range(nr_of_seqs):
        lista = random.choices([0,1], [0.5, 0.5], k=27)
        result.append(lista)
    return np.array(result)
test_out = generate_out(15)
train_out = generate_out(15)
valid_out = generate_out(15)'''



'def generate_out(nr_of_seqs):\n    result = []\n    for k in range(nr_of_seqs):\n        lista = random.choices([0,1], [0.5, 0.5], k=27)\n        result.append(lista)\n    return np.array(result)\ntest_out = generate_out(15)\ntrain_out = generate_out(15)\nvalid_out = generate_out(15)'

In [179]:
##########################################################
### Creating the .h5 file with the input for the model ###
##########################################################

with h5py.File('test3.h5', 'w') as data_file:
    data_file.create_dataset('test_in_region', data = test_in_region)
    data_file.create_dataset('test_in_seq', data = test_in_seq)
    data_file.create_dataset('test_name', data = test_name)
    data_file.create_dataset('test_out', data = test_out)
    data_file.create_dataset('train_in_region', data = train_in_region)
    data_file.create_dataset('train_in_seq', data = train_in_seq)
    data_file.create_dataset('train_out', data = train_out)
    data_file.create_dataset('valid_in_region', data = valid_in_region)
    data_file.create_dataset('valid_in_seq', data = valid_in_seq)
    data_file.create_dataset('valid_out', data = valid_out)
data_file.close()
# Testing whether it works:
data1 = h5py.File('test3.h5','r')
X_test_seq = np.array(data1['test_in_seq'])
print(data1.keys())
#print(data1['test_name'])


<KeysViewHDF5 ['test_in_region', 'test_in_seq', 'test_name', 'test_out', 'train_in_region', 'train_in_seq', 'train_out', 'valid_in_region', 'valid_in_seq', 'valid_out']>
